# A Quick Look at the MIMIC III Data Set



In [ ]:
import warnings
warnings.simplefilter("ignore")
from cdsutils.mutils import *
from cdsutils.sqlite import *


In [ ]:
!cp /home/shared/mimic3.db .

MIMIC III data are stored in a relational database. This is not an exploration of relational database theory or data modeling, but here is my novice quick description.

* Relational databases seek to achieve accurate data representation by eliminating (reducing)  data redundancies and thus the opportunities for data inconsistencies.

This is achieved by splitting data across **tables** and then **joining** the data back together when required.

### First we need to generate a connection to the MIMIC database

As you work through this notebook, you might occasionally get an error that looks something like this (although much longer):

```Python
OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

```

This just means that the connection with the database has timed out. All you need to do is come back up here and rerun the code below to get a new database connection.

In [ ]:
conn = get_mimic_connection()
conn.list_tables()
schema=None

## Let's take a look at the tables

>Before you can do anything, you have to understand tables. If you don't have a table, you have nothing to work on. The table is the standard unit of information in a relational database. Everything revolves around tables. Tables are composed of rows and columns. And while that sounds simple, the sad truth is that tables are not simple. (*The Definitive Guide to SQLite*, p. 80 [owens2006definitive})


Since I said data are split across tables, let's look at the tables in the MIMIC II demo database.

### Take a look at the Tables in the Database

In [ ]:
HTML(dlist(conn.list_tables(schema=schema), ncols=7, sort=True))

## MIMIC III is well documented

- You can read about each table [here](https://mimic.physionet.org/mimictables/).

- As an example we can look at [microbiologyevents](https://mimic.physionet.org/mimictables/microbiologyevents/)

## What are in the tables?
### Ibis Provides two ways to see the definitions of each table

1. `info()`
1. `schema()1
#### `info`

In [ ]:
t = conn.table("icustays")
t.info()


This is fairly ugly output, but tells us quite a bit about the table

- `Column`: This is the column name
- `Type`: This provides two pieces of information
    - The data type used to represent the data (e.g. `int32` (a 32 bit integer)
    - Whether the value is `nullable` (can be missing)
        - Example: `row_id` is represented with a 32 bit integer and CANNOT be missing
        - Example: `outtime` is represented with a `TimeStamp` and CAN be missing
- `Non-NULL #`: The number of rows in the table with non-NULL values for that column

#### `schema()`

`schema()` returns a dictionary-like object that provides the column names and the data tuype for the column, but does not provide any information about whether the value can be missing or not.

In [ ]:
itview(conn.table("labevents"))

In [ ]:
itview(t.projection(["subject_id", "icustay_id", "los"]))

In [ ]:
t.los.execute().describe()

In [ ]:
t.projection(["subject_id", "icustay_id", "los"]).filter(t.los > 35).execute()

In [ ]:
view_dict(t.schema())

In [ ]:
itview(conn.table("diagnoses_icd"))

## Things to notice

- `icd9_code` values are NOT ICD9 codes. They are references (foreign keys) to the definitions in `d_icd_diagnoses`
- `seq_num` is a ranking ("provides the order in which the ICD diagnoses relate to the patient") of the codes.


### Take a look at [`patients`](https://mimic.physionet.org/mimictables/patients/)



In [ ]:
view_table("patients", conn)

The [documentation](https://mimic.physionet.org/mimictables/patients/) tell us that this table links to `admission` and `icustays` vis the  `subject_id` value.

There are three different date of death columns. You can read about the differences and decide which value you would want to use.

- `NaT` represents a __missing time__.
- `gender`: `GENDER is the genotypical sex of the patient`

According to the WHO

>Humans are born with 46 chromosomes in 23 pairs. The X and Y chromosomes determine a person’s sex. Most women are 46XX and most men are 46XY. Research suggests, however, that in a few births per thousand some individuals will be born with a single sex chromosome (45X or 45Y) (sex monosomies) and some with three or more sex chromosomes (47XXX, 47XYY or 47XXY, etc.) (sex polysomies). In addition, some males are born 46XX due to the translocation of a tiny section of the sex determining region of the Y chromosome. Similarly some females are also born 46XY due to mutations in the Y chromosome. Clearly, there are not only females who are XX and males who are XY, but rather, there is a range of chromosome complements, hormone balances, and phenotypic variations that determine sex. (["Gender and Genetics"](https://www.who.int/genomics/gender/en/index1.html#:~:text=The%20X%20and%20Y%20chromosomes,47XYY%20or%2047XXY%2C%20etc.)))

So how many different genders are in the database?

We can use the `dictinct` method to get the unique values in a column:

In [ ]:
t_pat = conn.table("patients")
t_pat['gender'].distinct().execute(limit=None)

In [ ]:
t_pat.filter([t_pat.gender=='M']).count().execute(limit=None)

## How many total patients are there?

- `count()` counts the number of rows in the table
- A Note about execute: 

In [ ]:
t_pat.count().execute(limit=None)

### Look at [`admissions`](https://mimic.physionet.org/mimictables/admissions/) 

Because this table is wider than our display, you might want to __right click__ cell below and 
select "Create New View For Output". This will create a new embedded window that has horizontal scrolling.

In [ ]:
view_table("admissions", conn)

In addition to the admission, and discharge information, this table also contains demographic information.

### Examine [`prescriptions`](https://mimic.physionet.org/mimictables/prescriptions/)

For a patient being given medication (medication event), we would want to know things like who was the medicine given to, who gave it to them, what was the medicine, when was it given, etc.

Examining the `prescription` table we an see  the nature of a relational database


In [ ]:
t_pre = conn.table("prescriptions")
t_pre.info()

In [ ]:
display(view_dict(conn.table("prescriptions").schema()))
view_table("prescriptions", conn)

Similar to the `d_patients` table, there are values like dose that are provided explicity (e.g. 1350) and the unit of measurement for dose (`doseuom=Uhr` (unit per hour), but there are also a large number of values (recognized by the `id` component in the name) that are references to values defined in other tables. These ids are called **foreign keys.** Some of these include:

* `subect_id` that is defined in `d_patients`
* `cgid` that is defined `d_caregivers`
* `itemid` that is defined in `d_meditems`

Another important idea is represented in the column named `Null`. If the value is `YES`, that means the value is allowed to be missing. A value of `NO` means that a value must be provided. The issue of missing data will be very important for many applications.

To fully understand an entry, we need to join the values that are provided in a `medevent` entry with the values defined in the other tables referred to with the `id` values (foreign keys).

### Here is an example

* Joining information from `medevents`, `d_caregivers`, `d_careunits`, `d_meditems`, and `d_patients`
* I'm randomly selecting 50 cases

In [ ]:
display(view_dict(conn.table("chartevents").schema()))
view_table("chartevents", conn)

In [ ]:
t_chart = conn.table("chartevents")


In [ ]:
itview(conn.table("noteevents"))

### With a little bit of manipulation we can get the age

* Compute age
* Rename some of the columns, drop others
* **Note:** I'm doing all this in Python. If I were better at SQL, these steps could all be done within the query

In [ ]:
devents["age in years"] = devents.apply(lambda r: (r.charttime - r.dob).days/365.2425, axis=1)
devents2 = devents[["subject_id", "sex", "age in years", "charttime", "medname", 
                   "dose", "doseuom", "route", "cgid", "cglabel", "cuname"]]
devents2

### Some general comments

The database we are using is a *demo* database. Some tables are empty and others have values and foreign keys that I cannot figure. For example, I cannot find where the column `solutionid` in `medevents` is referring to, so we cannot decipher. There is not great documentation.

With completion of some simple online ethics training, you can get access to the complete version of [MIMIC (now MIMIC 3)](https://mimic.physionet.org/), which is well documented, has lots of example code available, and is one of the most valuable resources for clinical informatics research in the world.

### Questions

In [ ]:
for q in question_banks["qbank1"].values():
    display(q)